In [26]:
from scripts.find_feas_iti import _plot_check_feas_iti

_plot_check_feas_iti(rid=285)

In [44]:
import numpy as np
from src.utils import read_data, pd


def find_rids_with_same_final_train_in_all_itis():
    """
    Find rids in feas_iti_left.pkl where all feasible itineraries share the same final train (same train_id & alight_ts).

    Returns
    -------
    pd.DataFrame
        Subset of feas_iti_left containing only those rids whose all iti end with the same train.
    """
    df = read_data("feas_iti_left")

    # Keep only the last segment for each rid + iti_id
    # last_seg = df.sort_values("seg_id").groupby(["rid", "iti_id"]).tail(1)
    last_seg = df.groupby(["rid", "iti_id"]).last().reset_index()

    # 对每个 rid，聚合 (train_id, alight_ts) 为元组，统计唯一值数量
    last_seg["train_end"] = list(zip(last_seg["train_id"], last_seg["alight_ts"]))
    unique_end_count = last_seg.groupby("rid")["train_end"].nunique()

    # 筛选所有 iti 最终列车一致的 rid
    consistent_rids = unique_end_count[unique_end_count == 1].index

    return df[df["rid"].isin(consistent_rids)]


df = read_data("feas_iti_left")

# Keep only the last segment for each rid + iti_id
last_seg = df.groupby(["rid", "iti_id"]).last().reset_index()

# 对每个 rid，统计train_id唯一值数量
unique_end_count = last_seg.groupby("rid")["train_id"].nunique()

# 筛选所有 iti 最终列车一致的 rid
same_train_rids = unique_end_count[unique_end_count == 1].index

# 对于每个same_train_rid，随机选择一个iti_id的最后一行
last_alight = last_seg[last_seg["rid"].isin(same_train_rids)].groupby("rid").last()
last_alight

In [63]:
from src.globals import AFC
K_PV = read_data("pathvia", show_timer=False).values
# print(AFC[:3], "\n", K_PV[:3])

mask = np.isin(AFC[:, 0], same_train_rids)
filtered_AFC = AFC[mask]

egress_link = K_PV[len(K_PV) - 1 - np.unique(K_PV[:, 0][::-1], return_index=True)[1], :4]
path_id_node1 = {link[0]: link[2] for link in egress_link}
path_id_node2 = {link[0]: link[3] for link in egress_link}

last_alight['TS2'] = {record[0]: record[-1] for record in filtered_AFC}
last_alight["UID2"] = {record[0]: record[-2] for record in filtered_AFC}
last_alight["node1"] = last_alight["path_id"].map(path_id_node1)
last_alight["node2"] = last_alight["path_id"].map(path_id_node2)
print(last_alight[last_alight["node2"] != last_alight["UID2"]].shape[0])
last_alight

In [54]:
last_alight[["alight_ts", "TS2", "UID2"]]

In [59]:
K_PV[len(K_PV) - 1 - np.unique(K_PV[:, 0][::-1], return_index=True)[1], :4]

In [31]:
last_seg[last_seg["rid"].isin(unique_end_count[unique_end_count == 1].index)].groupby("rid").last()

In [24]:
last_seg[last_seg["rid"].isin([285, 286])]

In [25]:
df[(df['rid'] == 285) & (df['iti_id'].isin([1, 2]))]

## 多个feas_iti且最后一趟列车id相同的乘客

|           |    rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
|:----------|-------:|-------:|-----------:|-------:|---------:|---------:|----------:|
|93194730   | 1525794    |   2 | 1076103701  |     2 | 10301630   |  65071   |   65670|
|8610761    |  119014    |   6 | 1101102701  |     3 | 11002833   |  30045   |   30791|
|18578209   |  230428    |   6 | 1053108503  |     2 | 10401849   |  29715   |   30255|
|72855273   | 1321497    |   3 | 1110112802  |     1 | 10401844   |  60803   |   61537|
|105488240  | 1781873    |   2 | 1028106602  |     2 | 10401738   |  69396   |   69625|
|90908613   | 1485498    |   2 | 1023112302  |     2 | 10702370   |  64374   |   64780|
|91388272   | 1494286    |   1 | 1104111501  |     2 | 10702247   |  65070   |   65278|
|53621559   |  957712    |   6 | 1121102503  |     3 | 10401786   |  49174   |   50141|
|19767473   |  243641    |   1 | 1081103901  |     2 | 10702323   |  29509   |   31141|
|107978876  | 1880069    |   2 | 1129107002  |     1 | 10201226   |  73278   |   74154|
|106615427  | 1817923    |   1 | 1027110801  |     2 | 10301362   |  71471   |   73514|
|76737269   | 1359494    |   1 | 1099102101  |     2 | 10301421   |  64071   |   65266|
|100422001  | 1661221    |   5 | 1061108702  |     1 | 10100270   |  66558   |   66886|
|2408620    |   41409    |   2 | 1111106502  |     1 | 10401886   |  25727   |   25836|
|101105912  | 1675451    |   1 | 1110101301  |     1 | 10401914   |  66944   |   67031|
|92513072   | 1514419    |   3 | 1097110302  |     3 | 10200839   |  65302   |   65408|
|81121188   | 1412555    |   1 | 1061109504  |     2 | 10201199   |  63371   |   63480|
|108069499  | 1886032    |   1 | 1032108201  |     1 | 10200950   |  73568   |   75192|
|73737366   | 1331466    |   1 | 1086110901  |     2 | 10702414   |  61616   |   62107|
|48046191   |  811513    |   3 | 1076103602  |     3 | 10301265   |  42747   |   43084|